In [23]:
import numpy as np
import pandas as pd

In [2]:
# Assign spreadsheet filename to `file`
file = 'Все глобалы за существование ЧБ.xlsx'

# Load spreadsheet
xl = pd.ExcelFile(file)

In [40]:
# Print the sheet names
sheet_names_list = xl.sheet_names
df = None

for i in range(len(sheet_names_list)):
    # Load a sheet into a DataFrame by name: df1
    print(f'sheet_names_list={sheet_names_list[i]}')
    df_current = xl.parse(sheet_names_list[i])
    df_current = df_current.loc[:,['id','mesTExt','type','dialogid', 'Mdate','AgentEmail']]
    df_current = df_current.drop_duplicates()
    #df.head()

    df_current['Mdate'] = pd.to_datetime(df_current['Mdate'],format='%Y-%m-%d')
    df_current['Mdate'] = df_current['Mdate'].dt.date

    df_filter = df_current['type'].isin([1])
    df1 = df_current[df_filter]

    a = df1.groupby(['dialogid', 'Mdate'])['mesTExt'].transform(lambda answer : " ".join(str(x) for x in answer))
    df12= a.to_frame()

    df4 = pd.concat([df12, df1.loc[:,['dialogid','Mdate']]], axis=1)
    df4 = df4.drop_duplicates()

    df_filter = df_current['type'].isin([2])
    df5 = df_current[df_filter]
    df5 = df5.loc[:,['dialogid', 'Mdate','AgentEmail']]
    df5 = df5.drop_duplicates()

    df_req=pd.merge(df4, df5, on=['dialogid', 'Mdate'], how='inner')
    if i>0:
        df = pd.concat([df,df_req], axis=0, ignore_index=True, sort=False)
    else:
        df = df_req
    
df.to_csv("Заявки в чатбот.csv", encoding = 'utf-16', sep=";", index=False)   


sheet_names_list=май 18
sheet_names_list=Июнь 18
sheet_names_list=Июль 18
sheet_names_list=Авг 18
sheet_names_list=Сент 18
sheet_names_list=Окт 18
sheet_names_list=Ноя 18
sheet_names_list=Дек 18
sheet_names_list=Янв 19
sheet_names_list=Фев 19
sheet_names_list=Март 19
sheet_names_list=Апр 19
sheet_names_list=Май 19
sheet_names_list=Июнь 19
sheet_names_list=Июль 19
sheet_names_list=Авг 19
sheet_names_list=Сен 19
sheet_names_list=Окт 19
sheet_names_list=Ноя 19
sheet_names_list=Дек 19
sheet_names_list=Янв 20
sheet_names_list=Фев 20
sheet_names_list=Март 20
sheet_names_list=Апр 20


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35758 entries, 0 to 35757
Data columns (total 5 columns):
mesTExt                     35758 non-null object
dialogid                    35758 non-null int64
Mdate                       35758 non-null object
AgentEmail                  35758 non-null object
without_punctuation_text    35758 non-null object
dtypes: int64(1), object(4)
memory usage: 1.4+ MB


In [47]:
df.head()

,mesTExt,dialogid,Mdate,AgentEmail,without_punctuation_text
0,Привет Привет Привет Пока 1 End Привет 1 11 EN...,2792225,2018-05-19,Oksenich.a@dv-nevada.ru,Привет Привет Привет Пока 1 End Привет 1 11 EN...
1,Я тут 1 11 12 Сок добрый протек Сломан ноутбук...,2792602,2018-05-19,Oksenich.a@dv-nevada.ru,Я тут 1 11 12 Сок добрый протек Сломан ноутбук...
2,Привет Не работает ерп НЕ работает ЕРП Спасибо...,2792215,2018-05-21,Oksenich.a@dv-nevada.ru,Привет Не работает ерп НЕ работает ЕРП Спасибо...
3,Ерп Ерп 2 Ерп чинить сволочи будете или нет ? ...,2792602,2018-05-21,Oksenich.a@dv-nevada.ru,Ерп Ерп 2 Ерп чинить сволочи будете или нет ...
4,Висит ЕРП Висит ЕРП END Весит ERP End Висит пр...,2792225,2018-05-21,Oksenich.a@dv-nevada.ru,Висит ЕРП Висит ЕРП END Весит ERP End Висит пр...


In [48]:
import re

deleted_symols = '[\\\\\'[\]!"$%&()*+,-./:;<=>?@^_`{|}~«»\n]'
func = lambda text : re.sub(deleted_symols, ' ', text)
df["without_punctuation_text"] = df["mesTExt"].apply(func)
df.head()

,mesTExt,dialogid,Mdate,AgentEmail,without_punctuation_text
0,Привет Привет Привет Пока 1 End Привет 1 11 EN...,2792225,2018-05-19,Oksenich.a@dv-nevada.ru,Привет Привет Привет Пока 1 End Привет 1 11 EN...
1,Я тут 1 11 12 Сок добрый протек Сломан ноутбук...,2792602,2018-05-19,Oksenich.a@dv-nevada.ru,Я тут 1 11 12 Сок добрый протек Сломан ноутбук...
2,Привет Не работает ерп НЕ работает ЕРП Спасибо...,2792215,2018-05-21,Oksenich.a@dv-nevada.ru,Привет Не работает ерп НЕ работает ЕРП Спасибо...
3,Ерп Ерп 2 Ерп чинить сволочи будете или нет ? ...,2792602,2018-05-21,Oksenich.a@dv-nevada.ru,Ерп Ерп 2 Ерп чинить сволочи будете или нет ...
4,Висит ЕРП Висит ЕРП END Весит ERP End Висит пр...,2792225,2018-05-21,Oksenich.a@dv-nevada.ru,Висит ЕРП Висит ЕРП END Весит ERP End Висит пр...


In [49]:
df1 = df.drop(["mesTExt"], axis=1)
df1.to_csv("Заявки в чатбот (без пунктуации).csv", encoding = 'utf-16', sep=";", index=False) 

In [5]:
df = pd.read_csv('Заявки в чатбот (без пунктуации).csv', encoding = 'utf-16', sep=";")  
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35758 entries, 0 to 35757
Data columns (total 4 columns):
dialogid                    35758 non-null int64
Mdate                       35758 non-null object
AgentEmail                  35758 non-null object
without_punctuation_text    35683 non-null object
dtypes: int64(1), object(3)
memory usage: 1.1+ MB


In [7]:
func = lambda text : ' '.join([elem for elem in str(text).split(' ') if elem.isdigit() == False])   
df['without_numbers_text'] = df["without_punctuation_text"].apply(func)
df.head()

,dialogid,Mdate,AgentEmail,without_punctuation_text,without_numbers_text
0,2792225,2018-05-19,Oksenich.a@dv-nevada.ru,Привет Привет Привет Пока 1 End Привет 1 11 EN...,Привет Привет Привет Пока End Привет END ПРиве...
1,2792602,2018-05-19,Oksenich.a@dv-nevada.ru,Я тут 1 11 12 Сок добрый протек Сломан ноутбук...,Я тут Сок добрый протек Сломан ноутбук ерп вис...
2,2792215,2018-05-21,Oksenich.a@dv-nevada.ru,Привет Не работает ерп НЕ работает ЕРП Спасибо...,Привет Не работает ерп НЕ работает ЕРП Спасибо...
3,2792602,2018-05-21,Oksenich.a@dv-nevada.ru,Ерп Ерп 2 Ерп чинить сволочи будете или нет ...,Ерп Ерп Ерп чинить сволочи будете или нет За...
4,2792225,2018-05-21,Oksenich.a@dv-nevada.ru,Висит ЕРП Висит ЕРП END Весит ERP End Висит пр...,Висит ЕРП Висит ЕРП END Весит ERP End Висит пр...


In [8]:
df["tokenized_text"] = df['without_numbers_text']
df = df.drop(["without_numbers_text", "without_punctuation_text"], axis=1)
df.head()

,dialogid,Mdate,AgentEmail,tokenized_text
0,2792225,2018-05-19,Oksenich.a@dv-nevada.ru,Привет Привет Привет Пока End Привет END ПРиве...
1,2792602,2018-05-19,Oksenich.a@dv-nevada.ru,Я тут Сок добрый протек Сломан ноутбук ерп вис...
2,2792215,2018-05-21,Oksenich.a@dv-nevada.ru,Привет Не работает ерп НЕ работает ЕРП Спасибо...
3,2792602,2018-05-21,Oksenich.a@dv-nevada.ru,Ерп Ерп Ерп чинить сволочи будете или нет За...
4,2792225,2018-05-21,Oksenich.a@dv-nevada.ru,Висит ЕРП Висит ЕРП END Весит ERP End Висит пр...


In [9]:
n_words = len(set(" ".join(df["tokenized_text"]).split()))
n_words

44575

In [10]:
df["lowercase_text"] = df["tokenized_text"].apply(lambda text : text.lower())
df.head()

,dialogid,Mdate,AgentEmail,tokenized_text,lowercase_text
0,2792225,2018-05-19,Oksenich.a@dv-nevada.ru,Привет Привет Привет Пока End Привет END ПРиве...,привет привет привет пока end привет end приве...
1,2792602,2018-05-19,Oksenich.a@dv-nevada.ru,Я тут Сок добрый протек Сломан ноутбук ерп вис...,я тут сок добрый протек сломан ноутбук ерп вис...
2,2792215,2018-05-21,Oksenich.a@dv-nevada.ru,Привет Не работает ерп НЕ работает ЕРП Спасибо...,привет не работает ерп не работает ерп спасибо...
3,2792602,2018-05-21,Oksenich.a@dv-nevada.ru,Ерп Ерп Ерп чинить сволочи будете или нет За...,ерп ерп ерп чинить сволочи будете или нет за...
4,2792225,2018-05-21,Oksenich.a@dv-nevada.ru,Висит ЕРП Висит ЕРП END Весит ERP End Висит пр...,висит ерп висит ерп end весит erp end висит пр...


In [12]:
!pip install pymorphy2

In [11]:
df["lemmatized_text"] = df["lowercase_text"]

In [12]:
def to_lemmatize2(df):
    all_word_str = " ".join(df["lowercase_text"])
    all_word_list = all_word_str.split()
    all_unique_word = Series(all_word_list).unique()
    lemmatized_word_dict = {}
    lemmatizer = MorphAnalyzer()
    for word in all_unique_word:
        lemmatized_word_dict[word] = lemmatizer.normal_forms(word)[0]
    lemm_func = lambda text: ' '.join([lemmatized_word_dict[word] for word in text.split()])
    df['lemmatized_text'] = df['lowercase_text'].apply(lemm_func)
    return df, all_unique_word

In [13]:
%%time
from pandas import DataFrame, Series
from pymorphy2 import MorphAnalyzer

df, _ = to_lemmatize2(df)

Wall time: 13.1 s


In [14]:
df.head()

,dialogid,Mdate,AgentEmail,tokenized_text,lowercase_text,lemmatized_text
0,2792225,2018-05-19,Oksenich.a@dv-nevada.ru,Привет Привет Привет Пока End Привет END ПРиве...,привет привет привет пока end привет end приве...,привет привет привет пока end привет end приве...
1,2792602,2018-05-19,Oksenich.a@dv-nevada.ru,Я тут Сок добрый протек Сломан ноутбук ерп вис...,я тут сок добрый протек сломан ноутбук ерп вис...,я тут сок добрый протечь сломать ноутбук ерп в...
2,2792215,2018-05-21,Oksenich.a@dv-nevada.ru,Привет Не работает ерп НЕ работает ЕРП Спасибо...,привет не работает ерп не работает ерп спасибо...,привет не работать ерп не работать ерп спасибо...
3,2792602,2018-05-21,Oksenich.a@dv-nevada.ru,Ерп Ерп Ерп чинить сволочи будете или нет За...,ерп ерп ерп чинить сволочи будете или нет за...,ерп ерп ерп чинить сволочь быть или нет зависн...
4,2792225,2018-05-21,Oksenich.a@dv-nevada.ru,Висит ЕРП Висит ЕРП END Весит ERP End Висит пр...,висит ерп висит ерп end весит erp end висит пр...,висеть ерп висеть ерп end весить erp end висет...


LDA: коллекция должна быть представлена в виде списка списков, каждый внутренний список соответствует отдельному документу и состоит из его слов. Пример коллекции из двух документов: 

[["hello", "world"], ["programming", "in", "python"]]

Преобразуем наши данные в такой формат, а затем создадим объекты corpus и dictionary, с которыми будет работать модель.

In [15]:
lst = df['lemmatized_text'].tolist()
lst

['привет привет привет пока end привет end привет у я сломаться ноготь у я сломать ноготь у я сломать ноготь end у я сломаться ноутбук весь пропасть что делать спасибо end вообще чушь какой то помочь я срочно вообще чушь какой то помочь я срочно не работать ерп автозаказ привет привет добрый день сок добрый протечь сок добрый протечь',
 'я тут сок добрый протечь сломать ноутбук ерп висеть ерп висеть ерп висеть как оператор добавить номер телефон в личный кабинет парль',
 'привет не работать ерп не работать ерп спасибо пока ерп упасть напрочь',
 'ерп ерп ерп чинить сволочь быть или нет зависнуть ерп не загружаться заказ не проводиться это пользователь каждый надо быть делать end',
 'висеть ерп висеть ерп end весить erp end висеть програм ерп end не загружаться заказ end где мой заказ end не проводиться документ еклмна не проводиться документ ешка матрёшка end висеть ерп у клиент end',
 'висеть ерп висеть ерп end весить erp end висеть програм ерп end не загружаться заказ end где мой зака

In [16]:
texts = []
for i in range(len(lst)):    
    texts.append(lst[i].split(' '))
texts[35000]

['здравствовать',
 'срочно',
 'нужный',
 'помощь',
 'игод',
 'годовой',
 'собрание',
 'а',
 'я',
 'на',
 'удалёнка',
 'из',
 'скайп',
 'выбрасывать',
 'у',
 'я',
 'есть',
 'на',
 'тело',
 'я',
 'не',
 'мочь',
 'набрать',
 'я',
 'нужно',
 'на',
 'компьютер',
 'я',
 'ьула',
 'высутпать',
 'и',
 'я',
 'нужно',
 'показывать',
 'экран',
 'nan',
 'я',
 'только',
 'заходить',
 'я',
 'выбрасывать',
 'в',
 'линк',
 'по',
 'телефон',
 'я',
 'тоже',
 'не',
 'мочь',
 'подсоединить',
 'я',
 'не',
 'мочь',
 'ответить',
 'по',
 'тело',
 'я',
 'набрать',
 'по',
 'телефон',
 'я',
 'мочь',
 'только',
 'здесь',
 'я',
 'не',
 'знать',
 'что',
 'это',
 'мочь',
 'к',
 'я',
 'подключиться',
 'на',
 'телефон',
 'да',
 'я',
 'же',
 'гвоорать',
 'я',
 'нужный',
 'скайп',
 'именно',
 'на',
 'компьютер',
 'кинуть',
 'ссвлка',
 'я',
 'на',
 'почта',
 'пожалуйста',
 'ноутбук',
 'корпоративныца',
 'я',
 'не',
 'мочь',
 'установить',
 'nan',
 'nan',
 'куда',
 'далёкий',
 'запустить',
 'или',
 'установить',
 'просить'

In [17]:
from gensim import corpora, models

In [18]:
dictionary = corpora.Dictionary(texts)   # составляем словарь
corpus = [dictionary.doc2bow(text) for text in texts]  # составляем корпус документов

In [20]:
corpus[1]

[(6, 1),
 (7, 3),
 (11, 1),
 (16, 1),
 (18, 1),
 (20, 1),
 (27, 1),
 (28, 1),
 (29, 3),
 (30, 1),
 (31, 1),
 (32, 1),
 (33, 1),
 (34, 1),
 (35, 1),
 (36, 1),
 (37, 1),
 (38, 1)]

In [21]:
dictionary.token2id

{'end': 0,
 'автозаказ': 1,
 'весь': 2,
 'вообще': 3,
 'делать': 4,
 'день': 5,
 'добрый': 6,
 'ерп': 7,
 'какой': 8,
 'не': 9,
 'ноготь': 10,
 'ноутбук': 11,
 'пока': 12,
 'помочь': 13,
 'привет': 14,
 'пропасть': 15,
 'протечь': 16,
 'работать': 17,
 'сломать': 18,
 'сломаться': 19,
 'сок': 20,
 'спасибо': 21,
 'срочно': 22,
 'то': 23,
 'у': 24,
 'что': 25,
 'чушь': 26,
 'я': 27,
 'в': 28,
 'висеть': 29,
 'добавить': 30,
 'кабинет': 31,
 'как': 32,
 'личный': 33,
 'номер': 34,
 'оператор': 35,
 'парль': 36,
 'телефон': 37,
 'тут': 38,
 'напрочь': 39,
 'упасть': 40,
 'быть': 41,
 'зависнуть': 42,
 'загружаться': 43,
 'заказ': 44,
 'или': 45,
 'каждый': 46,
 'надо': 47,
 'нет': 48,
 'пользователь': 49,
 'проводиться': 50,
 'сволочь': 51,
 'чинить': 52,
 'это': 53,
 'erp': 54,
 'весить': 55,
 'где': 56,
 'документ': 57,
 'еклмна': 58,
 'ешка': 59,
 'клиент': 60,
 'матрёшка': 61,
 'мой': 62,
 'програм': 63,
 'заявка': 64,
 'ок': 65,
 'пожалуйста': 66,
 'прилетать': 67,
 'проблема': 68,
 

In [24]:
# обучение модель
np.random.seed(76543)
%time ldamodel = models.ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=50, passes=5, alpha=1.25, eta=1.25)

Wall time: 1min 41s


In [25]:
# выводим топы слов
for t, top_words in ldamodel.print_topics(num_topics=10, num_words=10):
    print("Topic", t, ":", top_words)

Topic 40 : 0.014*"nan" + 0.011*"день" + 0.011*"спасибо" + 0.009*"добрый" + 0.009*"нет" + 0.005*"не" + 0.004*"весь" + 0.002*"в" + 0.002*"просить" + 0.002*"здравствовать"
Topic 31 : 0.016*"nan" + 0.011*"спасибо" + 0.010*"добрый" + 0.008*"день" + 0.008*"нет" + 0.005*"не" + 0.003*"здравствовать" + 0.002*"весь" + 0.002*"в" + 0.002*"просить"
Topic 6 : 0.015*"nan" + 0.011*"добрый" + 0.010*"день" + 0.009*"спасибо" + 0.009*"весь" + 0.009*"нет" + 0.007*"не" + 0.003*"ок" + 0.003*"утро" + 0.002*"работать"
Topic 26 : 0.046*"не" + 0.033*"я" + 0.024*"работать" + 0.024*"мочь" + 0.020*"на" + 0.019*"зайти" + 0.017*"к" + 0.016*"и" + 0.015*"удалённый" + 0.015*"пароль"
Topic 24 : 0.021*"добрый" + 0.016*"день" + 0.012*"нет" + 0.011*"спасибо" + 0.007*"nan" + 0.006*"не" + 0.004*"весь" + 0.003*"утро" + 0.003*"на" + 0.002*"в"
Topic 43 : 0.052*"this" + 0.031*"a" + 0.030*"whatsapp" + 0.029*"system" + 0.029*"message" + 0.028*"to" + 0.027*"is" + 0.027*"client" + 0.026*"start" + 0.026*"with"
Topic 13 : 0.075*"спасиб

In [26]:
import copy
dictionary2 = copy.deepcopy(dictionary)

In [35]:
lst = []
for (key, value) in dictionary2.dfs.items():
    if value > 4000:
        lst.append(key)

In [39]:
dictionary2.filter_tokens(lst)

In [43]:
dict_size_after = len(dictionary2)
dict_size_after

22637

In [42]:
dict_size_before = len(dictionary)
dict_size_before

22709

In [44]:
corpus2 = [dictionary2.doc2bow(text) for text in texts]  # составляем корпус документов

In [45]:
# обучение модель
np.random.seed(76543)
%time ldamodel2 = models.ldamodel.LdaModel(corpus2, id2word=dictionary2, num_topics=50, passes=5, alpha=1.25, eta=1.25)

Wall time: 1min 24s


In [46]:
# выводим топы слов
for t, top_words in ldamodel2.print_topics(num_topics=10, num_words=10):
    print("Topic", t, ":", top_words)

Topic 44 : 0.002*"хорошо" + 0.001*"ок" + 0.001*"программа" + 0.001*"вечер" + 0.001*"большой" + 0.001*"1с" + 0.001*"задача" + 0.001*"выдавать" + 0.001*"пз" + 0.001*"закрывать"
Topic 43 : 0.035*"работа" + 0.032*"пожалуйста" + 0.025*"сервис" + 0.020*"обращение" + 0.019*"если" + 0.018*"дать" + 0.012*"отправить" + 0.012*"самбрать" + 0.012*"специалист" + 0.011*"группа"
Topic 34 : 0.001*"ок" + 0.001*"хорошо" + 0.001*"закрывать" + 0.001*"программа" + 0.001*"большой" + 0.001*"вечер" + 0.001*"выдавать" + 0.001*"задача" + 0.001*"пз" + 0.001*"отчёт"
Topic 17 : 0.002*"ок" + 0.002*"хорошо" + 0.001*"1с" + 0.001*"программа" + 0.001*"вечер" + 0.001*"отчёт" + 0.001*"выдавать" + 0.001*"большой" + 0.001*"закрывать" + 0.001*"пз"
Topic 6 : 0.002*"хорошо" + 0.001*"ок" + 0.001*"1с" + 0.001*"вечер" + 0.001*"закрывать" + 0.001*"программа" + 0.001*"выдавать" + 0.001*"большой" + 0.001*"пз" + 0.001*"очень"
Topic 31 : 0.002*"ок" + 0.002*"хорошо" + 0.001*"1с" + 0.001*"выдавать" + 0.001*"программа" + 0.001*"вечер" + 